# Part II: Model Development

In this part, we develop three unique pipelines for predicting backorder. We use the smart sample from Part I to fit and evaluate these pipelines. 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os, sys
import itertools
import numpy as np
import pandas as pd

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest

from sklearn.decomposition import FactorAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import Ridge

## Reload the smart sample here

In [4]:

# Reload your smart sampling from local file 
# ----------------------------------
import joblib

X_resampled = joblib.load('X_resampled.pkl')

y_resampled = joblib.load('y_resampled.pkl')


In [5]:
print("X, y shape:", X_resampled.shape, y_resampled.shape)

X, y shape: (21938, 20) (21938,)


## Normalize/standardize the data if required

In [6]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
X_resampled = scaler.fit_transform(X_resampled)

In [7]:
scaler2 = MinMaxScaler() 
X_resampled = scaler2.fit_transform(X_resampled)

In [8]:
print(X_resampled.mean(axis=0))
print(X_resampled.std(axis=0))

[7.00122400e-03 1.37303905e-01 3.28391807e-04 2.17321550e-04
 1.93543899e-04 1.86057414e-04 2.54580147e-04 2.08868552e-04
 1.88215774e-04 1.84169278e-04 9.97629684e-01 1.51309237e-03
 9.84685828e-01 9.87881197e-01 8.35161543e-04 8.15160908e-01
 9.99544170e-01 8.62886316e-01 2.69851399e-02 9.99954417e-01]
[0.01134625 0.11477363 0.00882239 0.00705728 0.00696846 0.00694757
 0.00717341 0.00699554 0.00695003 0.00694296 0.04862816 0.0186752
 0.11199358 0.09692022 0.01522159 0.38816698 0.02134531 0.34396733
 0.16203994 0.00675136]


## Split the data into Train/Test

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size = .25)

## Developing Pipeline

In this section, we design an operationalized machine learning pipeline, which includes:

* Anomaly detection
* Dimensionality Reduction
* Train a model

We are free to use any of the models that we learned in the past or use new models. 

* It is difficult to fit an anomaly detection method in the sklearn pipeline without writing custom codes. For simplicity, we avoid fitting an anomaly detection method within a pipeline. So we can create the workflow in two steps. 
    * Step I: fit an outlier with the training set
    * Step II: define a pipeline using a feature selection and a classification method. Then cross-validate this pipeline using the training data without outliers. 
        * Note: if your smart sample is somewhat imbalanced, you might want to change the scoring method in GridSearchCV (see the [doc](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)).

* Once we fit the pipeline, we identify the best model and give an unbiased evaluation using the test set that we created in Part II. For unbiased evaluation we report confusion matrix, precision, recall, f1-score, accuracy, and other measures if you like. 

(Optional) Those who are interested in writing custom codes for adding an outlier detection method into the sklearn pipeline, please follow this discussion [thread](https://stackoverflow.com/questions/52346725/can-i-add-outlier-detection-and-removal-to-scikit-learn-pipeline). 


**Note:** <span style='background:yellow'>We will be using Grid Search to find the optimal parameters of the pipelines.</span>

You can add more notebook cells or import any Python modules as needed.

### Your 1st pipeline 
  * Anomaly detection
  * Dimensionality reduction
  * Model training/validation
  
Add cells as needed. 

In [9]:
# Add anomaly detection code  (Question #E201)
# ----------------------------------

iso = IsolationForest(contamination=0.1).fit(X_train, y_train)

iso_outliers = iso.fit_predict(X_train)==-1
print(f"Num of outliers = {np.sum(iso_outliers)}")

Num of outliers = 1644


In [10]:
X_iso = X_train[~iso_outliers]
y_iso = y_train[~iso_outliers]

In [11]:
X_train_iso, X_test_iso, y_train_iso, y_test_iso = train_test_split(X_iso, y_iso, test_size=0.2)

In [12]:
clf = IsolationForest(contamination=0.1).fit(X_train_iso, y_train_iso)

In [13]:
# Add codes for feature selection and classification pipeline with grid search  (Question #E202)
# ----------------------------------

pca = PCA()

LR = LogisticRegression(max_iter=10000, tol=0.1)

pipe = Pipeline([
    ('scale', MinMaxScaler()), 
    ('PCA', PCA), 
    ('LR', LR)
])

In [14]:
tuned_parameters = {'C': [0.01, 0.1, 1, 10, 100]}

param_grid = {'pca__n_components': [5, 15, 30, 45, 70],
              'LR__C': [1e3, 5e3],        
              'LR__kernel': ['rbf']}
clf = GridSearchCV(LR, tuned_parameters, cv=5, n_jobs=2)

In [15]:
clf.fit(X_train_iso, y_train_iso)

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=10000, tol=0.1),
             n_jobs=2, param_grid={'C': [0.01, 0.1, 1, 10, 100]})

In [16]:
# Given an unbiased evaluation  (Question #E203)
# ----------------------------------
y_pred1 = clf.predict(X_test)
print(classification_report(y_test, y_pred1, zero_division = 1))

              precision    recall  f1-score   support

           0       0.63      0.53      0.58      2803
           1       0.58      0.68      0.63      2682

    accuracy                           0.60      5485
   macro avg       0.61      0.60      0.60      5485
weighted avg       0.61      0.60      0.60      5485



#### <center>Record the optimal hyperparameters and performance resulting from this pipeline.</center>

## <span style="background: yellow;">Commit your code!</span> 

### Your 2nd pipeline
  * Anomaly detection
  * Dimensionality reduction
  * Model training/validation

In [17]:
# Add anomaly detection code  (Question #E205)
# ----------------------------------

svm = OneClassSVM(kernel='rbf').fit(X_train, y_train)

svm_outliers = svm.fit_predict(X_train)==-1
print(f"Num of outliers = {np.sum(svm_outliers)}")

Num of outliers = 8227


In [18]:
X_svm = X_train[~svm_outliers]
y_svm = y_train[~svm_outliers]

In [19]:
X_train_svm, X_test_svm, y_train_svm, y_test_svm = train_test_split(X_svm, y_svm, test_size=0.2)

In [20]:
# Add codes for feature selection and classification pipeline with grid search  (Question #E206)
# ----------------------------------

selector = SelectKBest(chi2, k=4)

dt = DecisionTreeClassifier()

pipe = Pipeline(steps=[('selector', selector),
                       ('dt', dt)])

In [21]:
criterion = ['gini', 'entropy']
max_depth = [2,4,6,8,10,12]

param_grid = dict(dt__criterion=criterion,
                      dt__max_depth=max_depth)
clf2 = GridSearchCV(pipe, param_grid)

In [22]:
clf2.fit(X_train_svm, y_train_svm)

GridSearchCV(estimator=Pipeline(steps=[('selector',
                                        SelectKBest(k=4,
                                                    score_func=<function chi2 at 0x7f36c437f0d0>)),
                                       ('dt', DecisionTreeClassifier())]),
             param_grid={'dt__criterion': ['gini', 'entropy'],
                         'dt__max_depth': [2, 4, 6, 8, 10, 12]})

In [23]:
y_pred = clf2.predict(X_test)
print(classification_report(y_test, y_pred, zero_division = 1))

              precision    recall  f1-score   support

           0       0.57      0.87      0.69      2803
           1       0.71      0.32      0.44      2682

    accuracy                           0.60      5485
   macro avg       0.64      0.59      0.56      5485
weighted avg       0.64      0.60      0.57      5485



In [24]:
# Given an unbiased evaluation  (Question #E207)
# ----------------------------------


#### <center>Record the optimal hyperparameters and performance resulting from this pipeline.</center>

## <span style="background: yellow;">Commit your code!</span> 

### Your 3rd pipeline
  * Anomaly detection
  * Dimensionality reduction
  * Model training/validation

In [10]:
# Add anomaly detection code  (Question #E209)
# ----------------------------------

lof = LocalOutlierFactor(n_neighbors=2).fit(X_train, y_train)

lof_outliers = lof.fit_predict(X_train)==-1
print(f"Num of outliers = {np.sum(lof_outliers)}")

Num of outliers = 5008


In [11]:
X_lof = X_train[~lof_outliers]
y_lof = y_train[~lof_outliers]

In [12]:
X_train_lof, X_test_lof, y_train_lof, y_test_lof = train_test_split(X_lof, y_lof, test_size=0.2)

In [15]:
# Add codes for feature selection and classification pipeline with grid search  (Question #E210)
# ----------------------------------

from sklearn.feature_selection import RFECV

rfc = RandomForestClassifier(n_estimators=30, 
                                        random_state=42,
                                        class_weight="balanced") 

rfecv = RFECV(estimator=rfc, 
              step=1, 
              cv=5, 
              scoring = 'roc_auc')

pipeline  = Pipeline([('feature_sele',rfecv),
                      ('rfc',rfc)])

In [16]:
param_grid = RandomForestClassifier(n_estimators=10, 
                             random_state=42,
                             class_weight="balanced") 

In [17]:
clf = RandomForestClassifier(n_estimators=10, 
                             random_state=42,
                             class_weight="balanced") 

In [18]:
clf3 = GridSearchCV(clf, 
                      param_grid={'max_depth':[2,3]},
                      cv= 5, scoring = 'roc_auc')

In [19]:
clf3.fit(X_train_lof, y_train_lof)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(class_weight='balanced',
                                              n_estimators=10,
                                              random_state=42),
             param_grid={'max_depth': [2, 3]}, scoring='roc_auc')

In [20]:
y_pred3 = clf3.predict(X_test)
print(classification_report(y_test, y_pred3, zero_division = 1))

              precision    recall  f1-score   support

           0       0.80      0.85      0.83      2714
           1       0.85      0.79      0.82      2771

    accuracy                           0.82      5485
   macro avg       0.82      0.82      0.82      5485
weighted avg       0.82      0.82      0.82      5485



In [ ]:
# Given an unbiased evaluation  (Question #E211)
# ----------------------------------

#### <center>Record the optimal hyperparameters and performance resulting from this pipeline.</center>

## Compare these three pipelines and discuss your findings

## <span style="background: yellow;">Commit your code!</span> 

### Pickle the required pipeline/models for Part III.

In [33]:
import pickle

filename = 'my_final_project_model.pkl'

outfile = open(filename,'wb')

pickle.dump(y_pred3,outfile)

outfile.close()



You should have made a few commits so far of this project.  
**Definitely make a commit of the notebook now!**  
Comment should be: `Final Project, Checkpoint - Pipelines done`


# Save your notebook!
## Then `File > Close and Halt`